In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [4]:
from autoplan.trainer import ParserTrainer
from autoplan.dataset import PrelabeledDataset, build_synthetic_dataset
from autoplan.generator import ProgramGenerator
from autoplan.vis import plot_all_accuracy, plot_all_cm
from autoplan.token import PyretTokenizer, OCamlTokenizer

from grammars.rainfall.ocaml import Program
from grammars.rainfall.labels import GeneralRainfallLabels

from tqdm import tqdm_notebook as tqdm
import pandas as pd
import torch
import os

# device = torch.device('cuda:0')
device = torch.device('cpu')
REPO_DIR = os.path.expanduser('~/autoplan')

In [12]:
dataset_name = 'T1'
student_dataset = PrelabeledDataset.load(f'{REPO_DIR}/data/rainfall/{dataset_name}.pkl')

ValueError: 5 is not a valid GeneralRainfallLabels

In [7]:
generator = ProgramGenerator(grammar=Program())
program, choices, choice_options, labels = generator.generate()
print(program)

let  helper_name 
        (list_name : float list) (n : float) (num_elements : float)  : float =
    
    match list_name with
    | [] -> if num_elements > 0 then n /. num_elements else 0
        
    | head :: tail -> helper_name [] n num_elements
    | head :: tail -> helper_name tail (head + n) (num_elements + 1)

;;

let rec rainfall list_name =
    helper_name list_name 0. 0.;;
    


In [8]:
synth_dataset = build_synthetic_dataset(
    GeneralRainfallLabels,
    N=100,
    tokenizer=OCamlTokenizer(),
    generator=ProgramGenerator(grammar=Program()),
    vocab_index=student_dataset.vocab_index)

NameError: name 'student_dataset' is not defined

In [11]:
trainer = ParserTrainer(synth_dataset, device)
losses = []
train_eval = []
val_eval = []

In [14]:
for _ in tqdm(range(20)):
    losses.append(trainer.train_one_epoch())
    train_eval, val_eval = trainer.eval()
    train_evals.append(train_eval)
    val_evals.append(val_eval)

RuntimeError: index out of range at ../aten/src/TH/generic/THTensorEvenMoreMath.cpp:193

In [ ]:
pd.Series(losses).plot()

In [ ]:
plot_all_accuracy(train_evals)

In [ ]:
plot_all_accuracy(val_evals)

In [ ]:
plot_all_cm(val_evals[-1])